In [1]:
%tensorflow_version 2.x
import tensorflow as tf

TensorFlow 2.x selected.


In [0]:
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import re
import numpy as np
import os
import time
import json
from glob import glob
from PIL import Image
import pickle

In [3]:
cd /content/drive/My Drive/Colab Notebooks/insta/data

/content/drive/My Drive/Colab Notebooks/insta/data


## Downloading Data

In [4]:
"""annotation_folder = '/annotations/'
if not os.path.exists(os.path.abspath('.') + annotation_folder):
    annotation_zip = tf.keras.utils.get_file('captions.zip',
                                            cache_subdir=os.path.abspath('.'),
                                            origin = 'http://images.cocodataset.org/annotations/annotations_trainval2014.zip',
                                            extract = True
                                            )
    annotation_file = os.path.dirname(annotation_zip)+'/annotations/captions_train2014.json'
    os.remove(annotation_zip)

image_folder = '/train2014/'
if not os.path.exists(os.path.abspath('.') + image_folder):
    image_zip = tf.keras.utils.get_file('train2014.zip',
                                        cache_subdir=os.path.abspath('.'),
                                        origin = 'http://images.cocodataset.org/zips/train2014.zip',
                                        extract = True
                                        )
    PATH = os.path.dirname(image_zip) + image_folder
    os.remove(image_zip)
else:
    PATH = os.path.abspath('.') + image_folder"""

"annotation_folder = '/annotations/'\nif not os.path.exists(os.path.abspath('.') + annotation_folder):\n    annotation_zip = tf.keras.utils.get_file('captions.zip',\n                                            cache_subdir=os.path.abspath('.'),\n                                            origin = 'http://images.cocodataset.org/annotations/annotations_trainval2014.zip',\n                                            extract = True\n                                            )\n    annotation_file = os.path.dirname(annotation_zip)+'/annotations/captions_train2014.json'\n    os.remove(annotation_zip)\n\nimage_folder = '/train2014/'\nif not os.path.exists(os.path.abspath('.') + image_folder):\n    image_zip = tf.keras.utils.get_file('train2014.zip',\n                                        cache_subdir=os.path.abspath('.'),\n                                        origin = 'http://images.cocodataset.org/zips/train2014.zip',\n                                        extract = True\n         

In [0]:
annotation_folder = 'annotations/'
annotation_file = 'annotations/captions_train2014.json'
image_folder = 'train2014/'
PATH = image_folder

In [0]:
with open(annotation_file, 'r') as f:
    annotations = json.load(f)

all_captions = []
all_img_name_vector = []

for annotation in annotations['annotations']:
    caption = '<go> ' + annotation['caption'] + ' <eos>'
    image_id = annotation['image_id']
    full_coco_image_path = PATH + 'COCO_train2014_' + '%012d.jpg' % (image_id)

    all_img_name_vector.append(full_coco_image_path)
    all_captions.append(caption)


num_examples = 30000
train_captions = all_captions
img_name_vector = all_img_name_vector


In [7]:
len(train_captions), len(all_captions)

(414113, 414113)

In [0]:
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img

In [0]:
image_dataset = []
i = 0
while len(image_dataset) < num_examples:
    try:
        image_dataset.append(load_image(img_name_vector[i]))
        i += 1
        print(i, end='\r')
    except:
        pass

In [9]:
img_name_vector[:5]

['train2014/COCO_train2014_000000318556.jpg',
 'train2014/COCO_train2014_000000116100.jpg',
 'train2014/COCO_train2014_000000318556.jpg',
 'train2014/COCO_train2014_000000116100.jpg',
 'train2014/COCO_train2014_000000379340.jpg']

In [13]:
ls

COCO_train2014_000000000260.jpg  COCO_train2014_000000289845.jpg
COCO_train2014_000000000529.jpg  COCO_train2014_000000290544.jpg
COCO_train2014_000000000684.jpg  COCO_train2014_000000290754.jpg
COCO_train2014_000000000706.jpg  COCO_train2014_000000290911.jpg
COCO_train2014_000000001036.jpg  COCO_train2014_000000291018.jpg
COCO_train2014_000000001145.jpg  COCO_train2014_000000291780.jpg
COCO_train2014_000000001183.jpg  COCO_train2014_000000291915.jpg
COCO_train2014_000000001366.jpg  COCO_train2014_000000292053.jpg
COCO_train2014_000000001510.jpg  COCO_train2014_000000292088.jpg
COCO_train2014_000000001527.jpg  COCO_train2014_000000292518.jpg
COCO_train2014_000000001586.jpg  COCO_train2014_000000292829.jpg
COCO_train2014_000000001942.jpg  COCO_train2014_000000293277.jpg
COCO_train2014_000000002197.jpg  COCO_train2014_000000293314.jpg
COCO_train2014_000000002280.jpg  COCO_train2014_000000293339.jpg
COCO_train2014_000000002349.jpg  COCO_train2014_000000293366.jpg
COCO_train2014_0000000034

## Preprocess the captions

In [0]:
def calc_max_length(tensor):
    return max(len(t) for t in tensor)


In [0]:
top_k = 5000
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=top_k, oov_token="<unk>", filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
tokenizer.fit_on_texts(train_captions)
train_seqs = tokenizer.texts_to_sequences(train_captions)


In [0]:
tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'


In [0]:
decoder_inputs = [x[:-1] for x in train_seqs]
decoder_outputs = [x[1:] for x in train_seqs]

In [13]:
decoder_inputs[300], decoder_outputs[300]

([3, 16, 335, 19, 5, 1128, 1091, 2, 1840, 185, 86],
 [16, 335, 19, 5, 1128, 1091, 2, 1840, 185, 86, 4])

In [0]:
decoder_inputs = tf.keras.preprocessing.sequence.pad_sequences(decoder_inputs, padding='post')
decoder_outputs = tf.keras.preprocessing.sequence.pad_sequences(decoder_outputs, padding='post')

In [15]:
decoder_inputs[:2]

array([[   3,    2,  351,  687,    2,  280,    5,    2,   84,  339,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [   3,    2,   31, 2356,  112,    7,  137,    5,  159,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0]], dtype=int32)

In [16]:
decoder_outputs[:2]

array([[   2,  351,  687,    2,  280,    5,    2,   84,  339,    4,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [   2,   31, 2356,  112,    7,  137,    5,  159,    4,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0]], dtype=int32)

In [17]:
max_length = calc_max_length(train_seqs)
max_length

49

In [0]:
image_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')

In [19]:
image_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, None, None, 3 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, None, None, 3 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [0]:
for layer in image_model.layers:
    layer.trainable = False

image_model.trainable = False

In [0]:
c = image_model.output
c = tf.keras.layers.GlobalAveragePooling2D()(c)
c = tf.keras.layers.Dense(512, activation='relu')(c)


inputs = tf.keras.layers.Input(shape=(max_length))
x = tf.keras.layers.Embedding(top_k, max_length, mask_zero=True)(inputs)
x, c = tf.keras.layers.GRU(512, return_state=True, return_sequences=True)(x, initial_state=[c])
x, c = tf.keras.layers.GRU(512, return_state=True, return_sequences=True)(x, initial_state=[c])
x = tf.keras.layers.Dense(512, activation='relu')(x)
outputs = tf.keras.layers.Dense(top_k, activation='softmax')(x)

model = tf.keras.models.Model(inputs=[image_model.input, inputs], outputs=outputs)

In [0]:
model.compile(optimizer="adam", loss=tf.keras.losses.SparseCategoricalCrossentropy())

In [43]:
model.fit([image_dataset, decoder_inputs], decoder_outputs, epochs=1, validation_split=0.1, batch_size=16)

ValueError: ignored